 GOAL: Make a performance based dataset for the creation of model 2 mixed effects model in addition to the financial based dataset. 

In [2]:
import pandas as pd

In [3]:
# Step 1: Read all CSV files

df_defense = pd.read_csv('stats_squads_defense_for_2017-2018_to_2025-2026.csv')
print(f"Defense CSV: {len(df_defense)} rows, {len(df_defense.columns)} columns")

df_gca = pd.read_csv('stats_squads_gca_for_2017-2018_to_2025-2026.csv')
print(f"GCA CSV: {len(df_gca)} rows, {len(df_gca.columns)} columns")

df_keeper = pd.read_csv('stats_squads_keeper_for_2017-2018_to_2025-2026.csv')
print(f"Keeper CSV: {len(df_keeper)} rows, {len(df_keeper.columns)} columns")

df_standard = pd.read_csv('stats_squads_standard_for_2017-2018_to_2025-2026.csv')
print(f"Standard CSV: {len(df_standard)} rows, {len(df_standard.columns)} columns")


Defense CSV: 160 rows, 20 columns
GCA CSV: 160 rows, 20 columns
Keeper CSV: 160 rows, 22 columns
Standard CSV: 160 rows, 33 columns


In [4]:
# Step 2: Remove useless columns from Standard CSV (everything after Progression_PrgP)
# They are useless since they are the same stats per 90 minutes which we don't need for our analysis. 

# Find the index of 'Progression_PrgP'
if 'Progression_PrgP' in df_standard.columns:
    prgp_index = df_standard.columns.get_loc('Progression_PrgP')
    # Keep columns up to and including Progression_PrgP
    cols_to_keep = df_standard.columns[:prgp_index + 1]
    df_standard_cleaned = df_standard[cols_to_keep]
    print(f"Removed {len(df_standard.columns) - len(cols_to_keep)} columns from Standard CSV")
    print(f"Standard CSV after cleanup: {len(df_standard_cleaned)} rows, {len(df_standard_cleaned.columns)} columns")
else:
    print("Warning: 'Progression_PrgP' column not found!")
    df_standard_cleaned = df_standard

df_standard_cleaned.columns

Removed 10 columns from Standard CSV
Standard CSV after cleanup: 160 rows, 23 columns


Index(['Year', 'Squad', '# Pl', 'Age', 'Poss', 'Playing-Time_MP',
       'Playing-Time_Starts', 'Playing-Time_Min', 'Playing-Time_90s',
       'Performance_Gls', 'Performance_Ast', 'Performance_G+A',
       'Performance_G-PK', 'Performance_PK', 'Performance_PKatt',
       'Performance_CrdY', 'Performance_CrdR', 'Expected_xG', 'Expected_npxG',
       'Expected_xAG', 'Expected_npxG+xAG', 'Progression_PrgC',
       'Progression_PrgP'],
      dtype='object')

In [5]:
# Storing original row counts so we know how much data we have before merges. 

# Store original row counts for verification
original_rows = {
    'defense': len(df_defense),
    'gca': len(df_gca),
    'keeper': len(df_keeper),
    'standard': len(df_standard_cleaned)
}

original_rows

{'defense': 160, 'gca': 160, 'keeper': 160, 'standard': 160}

In [6]:
# Merge on Year and Squad (common columns)
# Start with standard as the base
combined_df = df_standard_cleaned.copy()
print(f"Starting with Standard CSV: {len(combined_df)} rows")

# Merge with defense
combined_df = combined_df.merge(
    df_defense,
    on=['Year', 'Squad'],
    how='outer',
    suffixes=('', '_defense')
)
print(f"After merging Defense: {len(combined_df)} rows")

# Merge with GCA
combined_df = combined_df.merge(
    df_gca,
    on=['Year', 'Squad'],
    how='outer',
    suffixes=('', '_gca')
)
print(f"After merging GCA: {len(combined_df)} rows")

# Merge with keeper
combined_df = combined_df.merge(
    df_keeper,
    on=['Year', 'Squad'],
    how='outer',
    suffixes=('', '_keeper')
)
print(f"After merging Keeper: {len(combined_df)} rows")


Starting with Standard CSV: 160 rows
After merging Defense: 160 rows
After merging GCA: 160 rows
After merging Keeper: 160 rows


In [7]:
# Check 1: Verify no duplicate rows
duplicates = combined_df.duplicated(subset=['Year', 'Squad']).sum()
print(f"\nCheck 1 - Duplicate (Year, Squad) pairs: {duplicates}")
if duplicates > 0:
    print("WARNING: Found duplicate entries!")


Check 1 - Duplicate (Year, Squad) pairs: 0


In [8]:
# Check 2: Verify all unique Year-Squad combinations are present
unique_combos = set()
for df_name, df in [('defense', df_defense), ('gca', df_gca), 
                     ('keeper', df_keeper), ('standard', df_standard_cleaned)]:
    for _, row in df.iterrows():
        unique_combos.add((row['Year'], row['Squad']))

combined_combos = set()
for _, row in combined_df.iterrows():
    combined_combos.add((row['Year'], row['Squad']))

print(f"\nCheck 2 - Unique (Year, Squad) combinations:")
print(f"  Expected (from all CSVs): {len(unique_combos)}")
print(f"  In combined dataframe: {len(combined_combos)}")
print(f"  Match: {len(unique_combos) == len(combined_combos)}")


Check 2 - Unique (Year, Squad) combinations:
  Expected (from all CSVs): 160
  In combined dataframe: 160
  Match: True


In [9]:
# Check 3: Verify row counts make sense
print(f"\nCheck 3 - Row count verification:")
print(f"  Original row counts: {original_rows}")
print(f"  Combined dataframe rows: {len(combined_df)}")
print(f"  Expected (max of originals): {max(original_rows.values())}")



Check 3 - Row count verification:
  Original row counts: {'defense': 160, 'gca': 160, 'keeper': 160, 'standard': 160}
  Combined dataframe rows: 160
  Expected (max of originals): 160


In [10]:
# Check 4: Check for missing data
print(f"\nCheck 4 - Missing data summary (BEFORE filling):")
null_counts = combined_df.isnull().sum()
cols_with_nulls = null_counts[null_counts > 0]
if len(cols_with_nulls) > 0:
    print(f"  Columns with missing values: {len(cols_with_nulls)}")
    print(f"  Total missing values: {null_counts.sum()}")
    print(f"\n  Details:")
    for col, count in cols_with_nulls.items():
        print(f"    - {col}: {count} missing values")
else:
    print("  No missing values detected!")

# Fill null values with 0
print(f"\nFilling all null values with 0...")
combined_df = combined_df.fillna(0)

# Verify no nulls remain
null_counts_after = combined_df.isnull().sum()
total_nulls_after = null_counts_after.sum()
print(f"Missing values after filling: {total_nulls_after}")


Check 4 - Missing data summary (BEFORE filling):
  Columns with missing values: 2
  Total missing values: 23

  Details:
    - Performance_CrdR: 20 missing values
    - PenaltyKicks_Save%: 3 missing values

Filling all null values with 0...
Missing values after filling: 0


In [11]:
combined_df.columns

Index(['Year', 'Squad', '# Pl', 'Age', 'Poss', 'Playing-Time_MP',
       'Playing-Time_Starts', 'Playing-Time_Min', 'Playing-Time_90s',
       'Performance_Gls', 'Performance_Ast', 'Performance_G+A',
       'Performance_G-PK', 'Performance_PK', 'Performance_PKatt',
       'Performance_CrdY', 'Performance_CrdR', 'Expected_xG', 'Expected_npxG',
       'Expected_xAG', 'Expected_npxG+xAG', 'Progression_PrgC',
       'Progression_PrgP', '# Pl_defense', '90s', 'Tackles-Tkl',
       'Tackles-TklW', 'Tackles-Def_3rd', 'Tackles-Mid_3rd', 'Tackles-Att_3rd',
       'Challenges-Tkl', 'Challenges-Att', 'Challenges-Tkl%',
       'Challenges-Lost', 'Blocks-Blocks', 'Blocks-Sh', 'Blocks-Pass',
       'Blocks-Int', 'Blocks-Tkl+Int', 'Blocks-Clr', 'Blocks-Err', '# Pl_gca',
       '90s_gca', 'SCA-SCA', 'SCA-SCA90', 'SCA_Types-PassLive',
       'SCA_Types-PassDead', 'SCA_Types-TO', 'SCA_Types-Sh', 'SCA_Types-Fld',
       'SCA_Types-Def', 'GCA_GCA', 'GCA_GCA90', 'GCA-Types_PassLive',
       'GCA-Types_Pass

In [12]:
# Display sample of combined data
print("\n" + "="*60)
print("SAMPLE OF COMBINED DATA (first 3 rows)")
print("="*60)
print(combined_df.head(3).to_string())


SAMPLE OF COMBINED DATA (first 3 rows)
   Year        Squad  # Pl   Age  Poss  Playing-Time_MP  Playing-Time_Starts  Playing-Time_Min  Playing-Time_90s  Performance_Gls  Performance_Ast  Performance_G+A  Performance_G-PK  Performance_PK  Performance_PKatt  Performance_CrdY  Performance_CrdR  Expected_xG  Expected_npxG  Expected_xAG  Expected_npxG+xAG  Progression_PrgC  Progression_PrgP  # Pl_defense   90s  Tackles-Tkl  Tackles-TklW  Tackles-Def_3rd  Tackles-Mid_3rd  Tackles-Att_3rd  Challenges-Tkl  Challenges-Att  Challenges-Tkl%  Challenges-Lost  Blocks-Blocks  Blocks-Sh  Blocks-Pass  Blocks-Int  Blocks-Tkl+Int  Blocks-Clr  Blocks-Err  # Pl_gca  90s_gca  SCA-SCA  SCA-SCA90  SCA_Types-PassLive  SCA_Types-PassDead  SCA_Types-TO  SCA_Types-Sh  SCA_Types-Fld  SCA_Types-Def  GCA_GCA  GCA_GCA90  GCA-Types_PassLive  GCA-Types_PassDead  GCA-Types_TO  GCA-Types_Sh  GCA-Types_Fld  GCA-Types_Def  # Pl_keeper  MP  Starts   Min  90s_keeper  Performance_GA  Performance_GA90  Performance_SoTA  Perf

In [14]:
# Save combined dataframe
output_file = 'combined_squad_stats_2017-2018-2025-2026.csv'
combined_df.to_csv(output_file, index=False)
print(f"\n✓ Combined dataframe saved to: {output_file}")
print(f"✓ Final dimensions: {len(combined_df)} rows × {len(combined_df.columns)} columns")


✓ Combined dataframe saved to: combined_squad_stats_2017-2018-2025-2026.csv
✓ Final dimensions: 160 rows × 79 columns
